### 调用chatgpt API识别一份报纸是否是民主党或者共和党

In [9]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

In [10]:
client = OpenAI(
    api_key='你的api-key',
)

def chat_gpt(newspaper_text,newspaper_name,newspaper_time):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
               "content": f"Based on the provided newspaper description, please determine the political affiliation of this newspaper during a specific period. Choose from the following five affiliations:\n\
                - **Republican Party**: Supports the Union, abolition of slavery, Black rights, and national unity. Advocates for economic modernization and nationwide infrastructure development, prioritizes federal authority over states' rights, and opposes slavery and secessionist movements.\n\
                - **Democratic Party**: Advocates for states' rights, supports Southern slavery and the Confederacy's secessionist stance, opposes excessive federal intervention, and believes states should have the right to decide their own economic and social policies.\n\
                - **Independent**: Does not lean toward any major political party, maintains a politically neutral stance, and generally does not prominently support or oppose specific parties or issues.\n\
                - **Whig Party**: Advocates for economic modernization, supports the banking system and infrastructure development, opposes excessive presidential power, takes a moderate stance, and focuses on social welfare and governance.\n\
                - **Other**: Does not fit any of the above political affiliations, or the provided information is insufficient to determine a specific affiliation.\n\n\
                Newspaper description:\n{newspaper_text}\n\n\
                Considering the newspaper’s historical background, publisher, time period, themes, and sponsors, please determine the political affiliation of {newspaper_name} during {newspaper_time}. Respond with only one of the five options above (e.g., 'Republican Party') without any additional explanation."

            }
        ]
    )
    return response.choices[0].message.content.strip()


In [56]:
# 使用正则表达式提取括号前后的内容
df['newspaper_name'] = df['title'].str.extract(r'^(.*?)\s*\(')[0]
df['newspaper_time'] = df['title'].str.extract(r'\)\s*(.*)$')[0]
df

,Unnamed: 0,newspaper_sn,state_icpsr,language,Change party,newspaper_introduction,"party(D,R,I)",title,pub_place,frequency,state,county,city,newspaper_name,newspaper_time
0,0,sn89051341,NaN,NaN,NaN,"\n\nDes Arc Citizen, Des Arc Semi-Weekly Citiz...",D,"Des Arc Semi-Weekly Citizen (Des Arc, Ark.) 18...","Des Arc, Ark. : J.C. Morrill",-? Semiweekly,Arkansas,Prairie,Des Arc,Des Arc Semi-Weekly Citizen,1861-1861
1,1,sn83045706,NaN,NaN,NaN,"\n\nThe North-Carolina Standard, Weekly North ...",D,"Weekly Standard (Raleigh, N.C.) 1858-1865","Raleigh, N.C. : William W. Holden, 1858-",-? Weekly,North Carolina,Wake,Raleigh,Weekly Standard,1858-1865
2,2,sn88053053,NaN,NaN,NaN,\n\nDelaware Inquirer\n\n\n\tThe Delaware Inqu...,D,"Delaware Inquirer (Wilmington, Del.) 1859-18??","Wilmington, Del. : James Montgomery",-? Weekly,Delaware,New Castle,Wilmington,Delaware Inquirer,1859-18??
3,3,sn85034457,NaN,NaN,NaN,"\n\nEaton Democrat, Preble County Democrat, Th...",D,"Eaton Weekly Democrat (Eaton, Ohio) 1866-1875","Eaton, Ohio : G.W. Mehaffey, -1875.",-? Weekly,Ohio,Preble,Eaton,Eaton Weekly Democrat,1866-1875
4,4,sn84027696,NaN,NaN,NaN,"\n\nDes Arc Citizen, Des Arc Semi-Weekly Citiz...",I,"Des Arc Weekly Citizen (Des Arc, Ark.) 1867-187?","Des Arc, Ark. : Poe & Mathews, 1867-",-? Weekly,Arkansas,Prairie,Des Arc,Des Arc Weekly Citizen,1867-187?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,940,sn85033295,NaN,NaN,NaN,"\n\nWatertown Republican, Watertown Weekly Lea...",NaN,"Watertown Republican (Watertown, Wis.) 1860-1906","Watertown, Wis. : J.W. Lawton, 1860-1906.",-? Weekly,Wisconsin,Dodge\nJefferson,Watertown,Watertown Republican,1860-1906
941,941,sn85035610,NaN,NaN,NaN,\n\nThe Lambertville Press\n\n\n\tThe Lambertv...,NaN,"The Lambertville Press (Lambertville, N.J.) 18...","Lambertville, N.J. : B.B. Blackwell",-? Weekly,New Jersey,Hunterdon,Lambertville,The Lambertville Press,1858-1???
942,942,sn84022278,NaN,NaN,NaN,"\n\nDallas Herald, Dallas Daily Herald and Dal...",NaN,"Dallas Herald (Dallas, Tex.) 1849-1873","Dallas, Tex. : Latimer & Swindells, 1849-1873.",-? Weekly,Texas,Dallas,Dallas,Dallas Herald,1849-1873
943,943,sn82015854,NaN,NaN,NaN,\n\nIowa Voter and Knoxville Journal\n\n\n\tTh...,NaN,"Knoxville Journal (Knoxville, Iowa) 1874-1986","Knoxville, Iowa : F.C. Barker, 1874-1986.",-? Weekly,Iowa,Marion,Knoxville,Knoxville Journal,1874-1986


In [57]:
dict_affiliation = {}
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    newspaper_sn = row['newspaper_sn']
    newspaper_text = row['newspaper_introduction']
    newspaper_name = row["newspaper_name"]
    newspaper_time = row["newspaper_time"]
    
    # 获取报纸的政治倾向
    affiliation = chat_gpt(newspaper_text,newspaper_name,newspaper_time)
    dict_affiliation[newspaper_sn] = affiliation

100%|████████████████████████████████████████████████████████████████████████████████| 945/945 [14:32<00:00,  1.08it/s]


In [1]:
# dict_affiliation

In [60]:
df_affiliation = pd.DataFrame(list(dict_affiliation.items()), columns=['newspaper_sn', 'party'])
df_affiliation

,newspaper_sn,party
0,sn89051341,Democratic Party
1,sn83045706,Democratic Party
2,sn88053053,Democratic Party
3,sn85034457,Democratic Party
4,sn84027696,Republican Party
...,...,...
940,sn85033295,Republican Party
941,sn85035610,Independent
942,sn84022278,Democratic Party
943,sn82015854,Republican Party


In [61]:
# 将 df_affiliation 和 df 按照 newspaper_sn 列合并
df_merged = pd.merge(df, df_affiliation, on='newspaper_sn', how='left')

In [62]:
df_merged 

,Unnamed: 0,newspaper_sn,state_icpsr,language,Change party,newspaper_introduction,"party(D,R,I)",title,pub_place,frequency,state,county,city,newspaper_name,newspaper_time,party
0,0,sn89051341,NaN,NaN,NaN,"\n\nDes Arc Citizen, Des Arc Semi-Weekly Citiz...",D,"Des Arc Semi-Weekly Citizen (Des Arc, Ark.) 18...","Des Arc, Ark. : J.C. Morrill",-? Semiweekly,Arkansas,Prairie,Des Arc,Des Arc Semi-Weekly Citizen,1861-1861,Democratic Party
1,1,sn83045706,NaN,NaN,NaN,"\n\nThe North-Carolina Standard, Weekly North ...",D,"Weekly Standard (Raleigh, N.C.) 1858-1865","Raleigh, N.C. : William W. Holden, 1858-",-? Weekly,North Carolina,Wake,Raleigh,Weekly Standard,1858-1865,Democratic Party
2,2,sn88053053,NaN,NaN,NaN,\n\nDelaware Inquirer\n\n\n\tThe Delaware Inqu...,D,"Delaware Inquirer (Wilmington, Del.) 1859-18??","Wilmington, Del. : James Montgomery",-? Weekly,Delaware,New Castle,Wilmington,Delaware Inquirer,1859-18??,Democratic Party
3,3,sn85034457,NaN,NaN,NaN,"\n\nEaton Democrat, Preble County Democrat, Th...",D,"Eaton Weekly Democrat (Eaton, Ohio) 1866-1875","Eaton, Ohio : G.W. Mehaffey, -1875.",-? Weekly,Ohio,Preble,Eaton,Eaton Weekly Democrat,1866-1875,Democratic Party
4,4,sn84027696,NaN,NaN,NaN,"\n\nDes Arc Citizen, Des Arc Semi-Weekly Citiz...",I,"Des Arc Weekly Citizen (Des Arc, Ark.) 1867-187?","Des Arc, Ark. : Poe & Mathews, 1867-",-? Weekly,Arkansas,Prairie,Des Arc,Des Arc Weekly Citizen,1867-187?,Republican Party
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,940,sn85033295,NaN,NaN,NaN,"\n\nWatertown Republican, Watertown Weekly Lea...",NaN,"Watertown Republican (Watertown, Wis.) 1860-1906","Watertown, Wis. : J.W. Lawton, 1860-1906.",-? Weekly,Wisconsin,Dodge\nJefferson,Watertown,Watertown Republican,1860-1906,Republican Party
941,941,sn85035610,NaN,NaN,NaN,\n\nThe Lambertville Press\n\n\n\tThe Lambertv...,NaN,"The Lambertville Press (Lambertville, N.J.) 18...","Lambertville, N.J. : B.B. Blackwell",-? Weekly,New Jersey,Hunterdon,Lambertville,The Lambertville Press,1858-1???,Independent
942,942,sn84022278,NaN,NaN,NaN,"\n\nDallas Herald, Dallas Daily Herald and Dal...",NaN,"Dallas Herald (Dallas, Tex.) 1849-1873","Dallas, Tex. : Latimer & Swindells, 1849-1873.",-? Weekly,Texas,Dallas,Dallas,Dallas Herald,1849-1873,Democratic Party
943,943,sn82015854,NaN,NaN,NaN,\n\nIowa Voter and Knoxville Journal\n\n\n\tTh...,NaN,"Knoxville Journal (Knoxville, Iowa) 1874-1986","Knoxville, Iowa : F.C. Barker, 1874-1986.",-? Weekly,Iowa,Marion,Knoxville,Knoxville Journal,1874-1986,Republican Party


In [66]:
# 删除 'Unnamed: 0' 列
df_merged  = df_merged .drop(columns=['Unnamed: 0'])

In [67]:
df_merged

,newspaper_sn,state_icpsr,language,Change party,newspaper_introduction,"party(D,R,I)",title,pub_place,frequency,state,county,city,newspaper_name,newspaper_time,party
0,sn89051341,NaN,NaN,NaN,"\n\nDes Arc Citizen, Des Arc Semi-Weekly Citiz...",D,"Des Arc Semi-Weekly Citizen (Des Arc, Ark.) 18...","Des Arc, Ark. : J.C. Morrill",-? Semiweekly,Arkansas,Prairie,Des Arc,Des Arc Semi-Weekly Citizen,1861-1861,Democratic Party
1,sn83045706,NaN,NaN,NaN,"\n\nThe North-Carolina Standard, Weekly North ...",D,"Weekly Standard (Raleigh, N.C.) 1858-1865","Raleigh, N.C. : William W. Holden, 1858-",-? Weekly,North Carolina,Wake,Raleigh,Weekly Standard,1858-1865,Democratic Party
2,sn88053053,NaN,NaN,NaN,\n\nDelaware Inquirer\n\n\n\tThe Delaware Inqu...,D,"Delaware Inquirer (Wilmington, Del.) 1859-18??","Wilmington, Del. : James Montgomery",-? Weekly,Delaware,New Castle,Wilmington,Delaware Inquirer,1859-18??,Democratic Party
3,sn85034457,NaN,NaN,NaN,"\n\nEaton Democrat, Preble County Democrat, Th...",D,"Eaton Weekly Democrat (Eaton, Ohio) 1866-1875","Eaton, Ohio : G.W. Mehaffey, -1875.",-? Weekly,Ohio,Preble,Eaton,Eaton Weekly Democrat,1866-1875,Democratic Party
4,sn84027696,NaN,NaN,NaN,"\n\nDes Arc Citizen, Des Arc Semi-Weekly Citiz...",I,"Des Arc Weekly Citizen (Des Arc, Ark.) 1867-187?","Des Arc, Ark. : Poe & Mathews, 1867-",-? Weekly,Arkansas,Prairie,Des Arc,Des Arc Weekly Citizen,1867-187?,Republican Party
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,sn85033295,NaN,NaN,NaN,"\n\nWatertown Republican, Watertown Weekly Lea...",NaN,"Watertown Republican (Watertown, Wis.) 1860-1906","Watertown, Wis. : J.W. Lawton, 1860-1906.",-? Weekly,Wisconsin,Dodge\nJefferson,Watertown,Watertown Republican,1860-1906,Republican Party
941,sn85035610,NaN,NaN,NaN,\n\nThe Lambertville Press\n\n\n\tThe Lambertv...,NaN,"The Lambertville Press (Lambertville, N.J.) 18...","Lambertville, N.J. : B.B. Blackwell",-? Weekly,New Jersey,Hunterdon,Lambertville,The Lambertville Press,1858-1???,Independent
942,sn84022278,NaN,NaN,NaN,"\n\nDallas Herald, Dallas Daily Herald and Dal...",NaN,"Dallas Herald (Dallas, Tex.) 1849-1873","Dallas, Tex. : Latimer & Swindells, 1849-1873.",-? Weekly,Texas,Dallas,Dallas,Dallas Herald,1849-1873,Democratic Party
943,sn82015854,NaN,NaN,NaN,\n\nIowa Voter and Knoxville Journal\n\n\n\tTh...,NaN,"Knoxville Journal (Knoxville, Iowa) 1874-1986","Knoxville, Iowa : F.C. Barker, 1874-1986.",-? Weekly,Iowa,Marion,Knoxville,Knoxville Journal,1874-1986,Republican Party


In [74]:
# 去除 frequency 列中的 '-?' 或 '?' 字符
df_merged['frequency'] = df_merged['frequency'].replace(r'[-?]?\s*\?', '', regex=True)
# 去除 party 列中的 '*' 字符
#df_merged['party'] = df_merged['party'].replace(r'\*', '', regex=True)

In [76]:
df_merged.to_csv("newspapr_location_party.csv",index = False)